In [1]:
import sys
import pandas as pd, time
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import holidays
#import trading_calendars as tc
sys.path.insert(1, 'Backtesting/')
import obv_for_target_date
import obv_for_target_date_original

In [2]:
def next_monday(date, weekday):
    d = datetime.strptime(date, '%Y-%m-%d')
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return str(d + timedelta(days_ahead)).split()[0]

def get_day_of_week(date):
    d = datetime.strptime(date, '%Y-%m-%d')
    return d.weekday()

def get_first_trading_day(date):
    if get_day_of_week(date) != 0:
        date = next_monday(date, 0)
    date = str(date).split()[0]
    return date

nyse = tc.get_calendar("XNYS")

FILE_PATH = r'/home/alekzandr/Documents/robinhood/Daily_Stock_Report/Stocks/'
start_time = datetime.now()
print("[-] Started at " + str(start_time))
i=0
Amount_of_API_Calls = 0
Stock_Failure = 0
Stocks_Not_Imported = 0
try:
    stock = 'DORM'  # Gets the current stock ticker
    temp = yf.Ticker(str(stock))
    Hist_data = temp.history(period="max")  # Tells yfinance what kind of data we want about this stock (In this example, all of the historical data)
    Hist_data.to_csv(FILE_PATH+stock+".csv")  # Saves the historical data in csv format for further processing later
    time.sleep(0.5)  # Pauses the loop for one seconds so we don't cause issues with Yahoo Finance's backend operations
    Amount_of_API_Calls += 1 
    Stock_Failure = 0
    i += 1  # Iteration to the next ticker
except ValueError:
    print("Yahoo Finance Backend Error, Attempting to Fix")  # An error occured on Yahoo Finance's backend. We will attempt to retreive the data again
    if Stock_Failure > 5:  # Move on to the next ticker if the current ticker fails more than 5 times
        i+=1
        Stocks_Not_Imported += 1
    Amount_of_API_Calls += 1
    Stock_Failure += 1
end_time = datetime.now()
print("[+] Completed at " + str(end_time))
print("[*] Total Time: " + str(end_time - start_time))
print("The amount of stocks we successfully imported: " + str(i - Stocks_Not_Imported))

In [3]:
us_holidays = holidays.US(years=2008)
'2009-01-01' in us_holidays
'2010-01-01' in us_holidays
'2011-01-01' in us_holidays
'2012-01-01' in us_holidays
'2013-01-01' in us_holidays
'2013-01-01' in us_holidays
'2014-01-01' in us_holidays
'2015-01-01' in us_holidays
'2016-01-01' in us_holidays
'2017-01-01' in us_holidays
'2018-01-01' in us_holidays
'2019-01-01' in us_holidays
'2020-01-01' in us_holidays
'2021-01-01' in us_holidays
holiday_list = us_holidays.get_named('day')

In [10]:
stocks_df, pct_change = obv_for_target_date_original.pct_change_past_week('2021-07-23', '2021-07-23', 20, 5, num_stocks=5)
stocks_df
print(stocks_df, pct_change)

Series([], Name: Stock, dtype: object)
Empty DataFrame
Columns: [Stock, Pct_Change]
Index: [] nan


In [5]:
results = []
funds = 100
contributions = 0
tax_flag = 0
start_date = '2018-01-01'
end_date = next_monday('2019-01-01', 0)
trading_day = get_first_trading_day(start_date)
current_day = None
print("First Trading Day for Backtest: " + trading_day)
while current_day != end_date:
    
    # Get next week's first trading day
    # 4 - Friday
    # 3 - Thursday
    # 2 - Wednesday
    # 0 - Monday
    next_trading_day = next_monday(trading_day, 0)
    
    # Uncomment for selling on Mondays
    if next_trading_day in us_holidays:
        next_trading_day = next_monday(next_trading_day, 1)
        
    
    # Uncomment for selling on Fridays
    #if nyse.is_session(pd.Timestamp(next_trading_day)) != True:
    #    next_trading_day = str(pd.Timestamp(next_trading_day) - timedelta(days=1)).split()[0]
    
    # Get number of days between next week's first trading day and this week's first trading day
    num_trading_days = np.busday_count(trading_day, next_trading_day, holidays = holiday_list)
    
    #print(next_trading_day)
    #print(num_trading_days)
    #print("Next Trading Day: " + next_trading_day)
    stocks_df, pct_change = obv_for_target_date_original.pct_change_past_week(trading_day, next_trading_day, 20, num_trading_days, num_stocks=5)
    results.append([trading_day, str(pct_change)])
    
    funds += contributions
    returns = funds * (pct_change/100)
    funds += returns
    print(trading_day, str(pct_change), funds)
    #print(stocks_df['Stock'].tolist())
    #break
    
    # Move window to next week
    trading_day = next_monday(trading_day, 0)
    if trading_day in us_holidays:
        trading_day = next_monday(trading_day, 1)
        
    current_day = trading_day
    
    

First Trading Day for Backtest: 2018-01-01
0    CAH
Name: Stock, dtype: object
2018-01-01 nan nan
0     AUY
1     GGB
2    VALE
3     CSX
4       F
Name: Stock, dtype: object
2018-01-08 4.837391414908102 nan
0     AUY
1     GGB
2    VALE
3     CSX
4     NOK
Name: Stock, dtype: object
2018-01-16 -2.672304463782709 nan


KeyboardInterrupt: 

In [39]:
stocks_df

,Stock,Pct_Change
0,AMC,64.252123
1,BBD,6.847816
2,F,8.763587


In [40]:
results_df = pd.DataFrame(results, columns=['Date', 'Percent_Change'])

In [41]:
results_df['SMA_10'] = results_df.iloc[:,1].rolling(window=10).mean()
results_df['SMA_20'] = results_df.iloc[:,1].rolling(window=20).mean()

In [42]:
results_df

,Date,Percent_Change,SMA_10,SMA_20
0,2021-01-04,4.628456566714295,NaN,NaN
1,2021-01-11,25.550777192407946,NaN,NaN
2,2021-01-19,5.511800100050902,NaN,NaN
3,2021-01-25,84.74746393994495,NaN,NaN
4,2021-02-01,-3.7052252615345345,NaN,NaN
5,2021-02-08,30.986362793721593,NaN,NaN
6,2021-02-16,-5.381349432276654,NaN,NaN
7,2021-02-22,-5.9802199604002695,NaN,NaN
8,2021-03-01,3.427196263678351,NaN,NaN
9,2021-03-08,0.8668155154235091,14.065208,NaN


In [43]:
results_df['EMA_10'] = results_df.iloc[:,1].ewm(span=10, adjust=False).mean()
results_df

,Date,Percent_Change,SMA_10,SMA_20,EMA_10
0,2021-01-04,4.628456566714295,NaN,NaN,4.628457
1,2021-01-11,25.550777192407946,NaN,NaN,8.432515
2,2021-01-19,5.511800100050902,NaN,NaN,7.901476
3,2021-01-25,84.74746393994495,NaN,NaN,21.873474
4,2021-02-01,-3.7052252615345345,NaN,NaN,17.222801
5,2021-02-08,30.986362793721593,NaN,NaN,19.725267
6,2021-02-16,-5.381349432276654,NaN,NaN,15.160428
7,2021-02-22,-5.9802199604002695,NaN,NaN,11.316673
8,2021-03-01,3.427196263678351,NaN,NaN,9.882223
9,2021-03-08,0.8668155154235091,14.065208,NaN,8.243058


In [44]:
results_df.to_csv('consolidated_3_stocks.csv')